In [1]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.7 MB/s eta 0:00:00


In [2]:
import random
import datetime
from faker import Faker

fake = Faker('es_AR')  # Faker configurado para Argentina

# Generar Proveedores
clasificaciones = ['preferido', 'regular', 'bajo rendimiento']
num_proveedores = 25
num_productos = 300
# Definir categorías y sus productos asociados
categorias_productos = {
  'Celulares': [
      'Samsung Galaxy S23', 'iPhone 14', 'Motorola Edge', 'Xiaomi Redmi Note', 'Huawei P50'
  ],
  'Computadoras': [
      'Dell Inspiron', 'HP Pavilion', 'Apple MacBook Pro', 'Lenovo ThinkPad', 'Acer Aspire'
  ],
  'Tablets': [
      'Apple iPad Pro', 'Samsung Galaxy Tab', 'Microsoft Surface Go', 'Huawei MatePad', 'Lenovo Tab P11'
  ],
  'Gaming': [
      'Sony PlayStation 5', 'Microsoft Xbox Series X', 'Nintendo Switch', 'Razer Gaming Laptop', 'Alienware Desktop'
  ],
  'Oficina': [
      'Impresora HP LaserJet', 'Canon Pixma', 'Logitech Mouse', 'OfficeDesk Chair', 'Ergonomic Keyboard'
  ]
}
categorias = list(categorias_productos.keys())


proveedores = []
for _ in range(num_proveedores):
    proveedores.append((
        fake.company(),
        fake.name(),
        fake.address().replace('\n', ', '),
        fake.city(),
        fake.postcode(),
        'Argentina',
        fake.phone_number(),
        random.choice(clasificaciones),
        random.randint(0,1)
    ))

# Generar Clientes
num_clientes = 150
clientes = []
for _ in range(num_clientes):
    clientes.append((
        fake.first_name(),
        fake.last_name(),
        fake.unique.email(),
        fake.phone_number(),
        fake.address().replace('\n', ', '),
        fake.city(),
        fake.postcode(),
        'Argentina'
    ))

# Generar Productos
productos = []
for _ in range(num_productos):
    categoria = random.choice(categorias)
    nombre_producto = random.choice(categorias_productos[categoria])

    # Definir rangos de precios y costos por categoría
    if categoria == 'Celulares':
        precio = round(random.uniform(200000.00, 1500000.00), 2)  # Precios en ARS
        costo = round(precio * random.uniform(0.6, 0.8), 2)
    elif categoria == 'Computadoras':
        precio = round(random.uniform(500000.00, 1500000.00), 2)
        costo = round(precio * random.uniform(0.5, 0.75), 2)
    elif categoria == 'Tablets':
        precio = round(random.uniform(100000.00, 1200000.00), 2)
        costo = round(precio * random.uniform(0.6, 0.8), 2)
    elif categoria == 'Gaming':
        precio = round(random.uniform(600000.00, 1800000.00), 2)
        costo = round(precio * random.uniform(0.5, 0.7), 2)
    elif categoria == 'Oficina':
        precio = round(random.uniform(5000.00, 50000.00), 2)
        costo = round(precio * random.uniform(0.4, 0.6), 2)
    else:
        precio = round(random.uniform(1000.00, 50000.00), 2)
        costo = round(precio * random.uniform(0.5, 0.8), 2)

    proveedor_id = random.randint(1, num_proveedores)

    productos.append((
        nombre_producto,
        categoria,
        precio,
        costo,
        proveedor_id
    ))

# Generar Moneda
num_moneda = 20
monedas = ['Dólar']
moneda_registros = []
for i in range(num_moneda):
    moneda_registros.append((
        round(random.uniform(800.5, 1100.5), 4),
        random.choice(monedas),
        fake.date_between(start_date='-1y', end_date='today')
    ))

# Generar Envios
transportistas = ['Transportista A', 'Transportista B', 'Transportista C', 'Transportista D']
estados_envio = ['En tránsito', 'Entregado', 'Pendiente', 'Retrasado']
num_envios = 50
envios = []
for _ in range(num_envios):
    envios.append((
        fake.date_time_between(start_date='-1y', end_date='now'),
        random.choice(transportistas),
        random.choice(estados_envio)
    ))

# Generar Devoluciones
num_devoluciones = 15
estados_devolucion = ['Procesada', 'Pendiente', 'Rechazada']
devoluciones = []
for _ in range(num_devoluciones):
    devoluciones.append((
        random.randint(1, num_productos),
        fake.date_time_between(start_date='-1y', end_date='now'),
        fake.sentence(nb_words=6),
        random.choice(estados_devolucion)
    ))

# Generar Inventario
num_inventario = 300
inventario = []
for producto_id in range(1, num_inventario + 1):
    inventario.append((
        producto_id,
        random.randint(0, 1000),
        fake.date_time_between(start_date='-1y', end_date='now')
    ))

# Función para generar sentencias INSERT con manejo adecuado de comillas
def generate_insert(table, columns, values):
  insert_stmt = f"INSERT INTO {table} ({', '.join(columns)}) VALUES\n"
  value_lines = []
  for value in values:
      formatted_values = []
      for v in value:
          if isinstance(v, str):
              # Escapar comillas simples duplicándolas para SQL
              sanitized_value = v.replace("'", "''")
              formatted_values.append(f"'{sanitized_value}'")
          elif isinstance(v, datetime.datetime):
              # Formatear datetime sin microsegundos
              formatted_date = v.strftime('%Y-%m-%d %H:%M:%S')
              formatted_values.append(f"'{formatted_date}'")
          elif isinstance(v, (int, float)):
              formatted_values.append(str(v))
          elif isinstance(v, bool):
              formatted_values.append(str(int(v)))
          else:
              # Para tipos de datos no manejados, convertir a string y escapar comillas
              sanitized_value = str(v).replace("'", "''")
              formatted_values.append(f"'{sanitized_value}'")
      value_lines.append("(" + ", ".join(formatted_values) + ")")
  insert_stmt += ",\n".join(value_lines) + ";\n"
  return insert_stmt

# Generar los scripts
script = ""

# Proveedores
script += generate_insert(
    'Proveedores',
    ['NombreProveedor', 'NombreContacto', 'Direccion', 'Ciudad', 'Estado', 'CodigoPostal', 'Pais', 'Telefono', 'Clasificacion', 'EsProveedorCertificado'],
    proveedores
)

# Clientes
script += generate_insert(
    'Clientes',
    ['Nombre', 'Apellido', 'CorreoElectronico', 'Telefono', 'Direccion', 'Ciudad', 'Estado', 'CodigoPostal', 'Pais'],
    clientes
)

# Productos
script += generate_insert(
    'Productos',
    ['NombreProducto', 'Categoria', 'Precio', 'Costo', 'ProveedorID'],
    productos
)

# Moneda
script += generate_insert(
    'Moneda',
    ['Cotizacion', 'Moneda', 'Fecha'],
    moneda_registros
)

# Envios
script += generate_insert(
    'Envios',
    ['FechaEntrega', 'Transportista', 'EstadoEnvio'],
    envios
)

# Devoluciones
script += generate_insert(
    'Devoluciones',
    ['ProductoID', 'FechaDevolucion', 'MotivoDevolucion', 'EstadoDevolucion'],
    devoluciones
)

# Inventario
script += generate_insert(
    'Inventario',
    ['ProductoID', 'CantidadDisponible', 'FechaActualizacion'],
    inventario
)

# Guardar el script en un archivo
with open('insert_datamineria.sql', 'w', encoding='utf-8') as f:
    f.write(script)

print("Script 'insert_datamineria.sql' generado con éxito.")

Script 'insert_datamineria.sql' generado con éxito.
